### Non-Negative Matrix Factorization

A lab experience on NMF with the Olivetti Faces Dataset

Code resources:
- [Scikit Learn: Non-Negative Matrix Factorization (NMF)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html#nmf)
- [Scikit Learn tutorial: Face dataset decompositions](https://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html#sphx-glr-auto-examples-decomposition-plot-faces-decomposition-py)
- [Geeks for Geeks: Face dataset decompositions in Scikit Learn](https://www.geeksforgeeks.org/faces-dataset-decompositions-in-scikit-learn/)

### Import libraries

In [21]:
import numpy as np

from matplotlib import pyplot as plt
from sklearn.decomposition import NMF
from sklearn.datasets import fetch_olivetti_faces

### Import and prepare data

In [22]:
# for reproducibility
RANDOM_STATE = 42

In [23]:
# Fetch and store data object
data_obj = fetch_olivetti_faces(shuffle=True, random_state=RANDOM_STATE)

# Array with serialised (flattened) images
flat_images = data_obj.data

n_samples, n_features = flat_images.shape

# Array with 2D images
images = data_obj.images

### Plot settings

In [24]:
# Plot grid rows
ROWS = 6

# Plot grid columns
COLS = 6

N_IMAGES = ROWS * COLS

IMAGE_SHAPE = (64, 64)

### Plot sample of original images

In [ ]:
def plot_gallery(
        title: str,
        images_array: np.ndarray,
        n_cols: int = COLS,
        n_rows: int = ROWS,
        image_shape: tuple = IMAGE_SHAPE
) -> None:
    
    '''
    Plot a sample of images from the Olivetti faces dataset.

    :param title: Plot title
    :param images_array: Array with flattened images
    :param n_cols: Number of plot grid columns
    :param n_rows: Number of plot grid rows
    :param image_shape: Required image shape for plotting
    '''
    
    # Init plot grid
    fig, axes = plt.subplots(
        nrows = n_rows, ncols=n_cols,
        figsize = (2.0 * n_cols, 2.3 * n_rows),
        facecolor = 'white',
        constrained_layout = True
    )

    # Plot grid configuration
    fig.set_constrained_layout_pads(w_pad=0.01, h_pad=0.02, hspace=0, wspace=0)
    fig.set_edgecolor("black")
    fig.suptitle(title, size=16)

    # Plot a sample of images
    image = None  # Init image variable
    
    for ax, img in zip(axes.flat, images_array):
        v_max = max(img.max(), -img.min())
        
        image = ax.imshow(
            img.reshape(image_shape),
            cmap="gray", interpolation="nearest",
            vmin=-v_max, vmax=v_max
        )

        ax.axis("off")

    fig.colorbar(
        image, ax=axes, orientation="horizontal",
        shrink=0.99, aspect=40, pad=0.01
    )

    plt.show()

plot_gallery("A selection of Olivetti faces", flat_images[:N_IMAGES])

Set the dimensinality of the factorization

This is a hyperparamenter and can be changed to see how it affects the results.

In [26]:
LATENT_FEATURES = 36

In [ ]:
# Init NMF model: Set number of components equal to image sample size for plotting
nmf_model = NMF(
    n_components = LATENT_FEATURES,
    max_iter = 500,
    tol = 0.001,
    random_state = RANDOM_STATE,
    # Lee-Seung's Multiplicative Update method (it's default but we notice it)
    solver = 'mu'
)

# Fit model
nmf_model.fit(flat_images)

### Get array H (basic facial features)

In [ ]:
H = nmf_model.components_

H.shape

### Plot H

In [ ]:
# Make sure that n_rows * n_cols = LATENT_FEATURES
plot_gallery(
    f"Latent features ({LATENT_FEATURES})",
    H, 
    n_cols = 6,
    n_rows = 6
)

### Get weights W (transformed images)

In [ ]:
# Array W
W = nmf_model.transform(flat_images)

W.shape

### Plot W

In [ ]:
plot_gallery("Transformed Images", 
             W[:N_IMAGES], 
             image_shape=(6, 6)
)

### Plot the re-constructed images

In [ ]:
reconstructed_images = np.dot(W, H)

plot_gallery('Reconstructed Images', reconstructed_images[:N_IMAGES])

Now work on the hyperparameters to find the threshold value that gives interpretable results but also a good reconstructed image.

## Root Mean Squared Error

In [ ]:
np.sqrt(np.mean(np.power(flat_images - reconstructed_images, 2)))